In [3]:
pip install psycopg2-binary pandas 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 139.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 272.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import psycopg2
from config import config
import pandas as pd

def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()
 
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
      
        # create a cursor
        cur = conn.cursor()
        
   # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
 
        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
       # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
 
 
if __name__ == '__main__':
    connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 17.4 (Debian 17.4-1.pgdg120+2) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)
Database connection closed.


In [5]:
def getTables():
    """Return a list of table names in the database."""
    conn = None
    tables = []
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute("""SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'""")
        
        tables = [table[0] for table in cur.fetchall()]  # Convert tuples to a list of strings
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Database error: {error}")
    finally:
        if conn is not None:
            conn.close()
    
    return tables  # Return the list

# Now call the function and iterate over the result
tables = getTables()

for t in tables:
    print(t)


movies
actors
genre
movie_ids
actor_movies


In [7]:
def getColumns(schema, table):
    """Return the column names of the given table."""
    conn = None
    column_names = []
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()

        # Debug: Print the final SQL query
        sql_command = f"""
        SELECT column_name FROM information_schema.columns 
        WHERE table_schema = '{schema}' AND table_name = '{table}'
        """

        cur.execute(sql_command)
        column_names = [row[0] for row in cur.fetchall()]

        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Database error: {error}")
    finally:
        if conn is not None:
            conn.close()
    
    return column_names

for t in tables:
    table = t
    print("Table: %s" % table)
    columns = getColumns("public", table)
    print(f"{table}: {columns}")


Table: movies
movies: ['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'origin_country', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count']
Table: actors
actors: ['adult', 'id', 'name', 'popularity']
Table: genre
genre: ['id', 'name']
Table: movie_ids
movie_ids: ['id', 'adult', 'original_title', 'popularity', 'video']
Table: actor_movies
actor_movies: ['actor_id', 'movie_id']
